## COLAB환경에서 DARKNET 실습
### YOLO_Project를 위해서는 GPU 필수
#### YOLO : You Only Look Once 
이번 실습에서는 YOLO를 자신이 원하는 조건에 맞게  Custom

# Colab 사용 이유
* Darknet 사용해야 하는데 설치조건이 까다로운데 COLAB을 사용하면 공통적인 환경에서 실습가능 (예전에는 Linux만 지원을 했으나 요즘에는 Mac,Windows에도 지원해줌)
* GPU연산을 통해서도 상당한 시간이 걸리는데 이걸 CPU연산을 하면 상당히 오랜시간이 걸림

# Colab
* 12시간마다 정보들이 휘발(무료버전)
    * 구글드라이브로 정보들을 저장해야함
    * Git을 이용 
* GPU 연산 가능
* Linux 환경
* Data 시각화가능




# 개발환경 만들기
1. GPU Setup
런타임 → 런타임 유형 변경 → 하드웨어 가속기(CPU) → 하드웨어 가속기(GPU)로 설정

In [ ]:
런타임 → 런타임 유형 변경 → 하드웨어 가속기(CPU) → 하드웨어 가속기(GPU)로 설정

2. **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls 
#확인

3. **Install CUDA related modules**
* 대표적인 GPU 

In [ ]:
!/usr/local/cuda/bin/nvcc --version
## 현재 CUDA버전 확인 현재 11.1
''' NVIDIA 홈페이지에서 확인
https://developer.nvidia.com/rdp/cudnn-download
회원가입 후 다운로드
cuDNN : CUDA의 소프트웨어
deep neural networks를 사용하기 위해 
자신의 CUDA 버전과 맞는 버전 다운
'''
!arch 
# 리눅스 버전확인
'''
다운받은 파일을 구글 Drive의 darknet이란 폴더를 만들고 그 안에 cuDNN 폴더를 만들어서 옮겨놓는다.
연결된 경로복사 후 압축해제
'''

In [ ]:
!tar -xzvf drive/MyDrive/Kobot/YOLO/darknet/cuDNN/cudnn-11.1-linux-x64-v8.0.5.39.tgz -C /usr/local/
## 압축해제
!chmod a+r /usr/local/cuda/include/cudnn.h

In [ ]:
!cat /usr/local/cuda/include/cudnn.h
# 설치확인

**4. Install Draknet **
* CUDA와 C와 기본으로 하며 빠르고 쉽다. 
* DarkNet install https://pjreddie.com/darknet/install/
* Colab에서 사용하기 편하게 바꾼 코드 실행

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

In [ ]:
!ls
# Clone 내용 확인

In [ ]:
!git checkout feature/google-colab

In [ ]:
#Compile DarkNet (매번 할 필요없이 1회만 하면 된다.)
# Makefile 수정단계
%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile


In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive/Kobot/YOLO/darknet/bin'):
    os.makedirs('/content/drive/MyDrive/Kobot/YOLO/darknet/bin')
# 만약 bin이라는 폴더가 없다면 bin이라는 폴더를 생성하는 코드

In [ ]:
!make

In [ ]:
!cp -r ./darknet /content/drive/MyDrive/Kobot/YOLO/darknet/bin/darknet
# 드라이브에 복사 
# 컴파일 과정없이 다음부터는 불러오면 된다 아래 참조

In [ ]:
# Use compiled darknet in your Google Drive after the first compile
# 1. Makes a darknet dir
# !mkdir darknet
# %cd darknet

# 2. Copy the Darknet compiled version to the VM
# !cp /content/drive/MyDrive/Kobot/YOLO/darknet/bin/darknet ./darknet

# 3. Set execution permissions
# !chmod +x ./darknet

5. Colab 에서 Yolo 사용

In [ ]:
#download files

def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    img = cv2.imread(path)
    height , width = img.shape[:2]
    resized_img = cv2.resize(img, (3*width, 3*height),interpolation = cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18,10)
    plt.axis("off")
    #plt.rcParams['figure.figsize'] = [10,5]
    plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
    plt.show()

def upload():
    from google.colab import files
    uploaded = files.upload()
    for name, data in uploaded.items():
        with open(name, "wb") as f:
            f.write(data)
            print("saved file", name)
def download(path):
    from google.colab import files
    files.download(path)

In [ ]:
# Darknet 에서 미리 학습된 데이터를 구글드라이브에서 가져와서 복사
# https://pjreddie.com/darknet/yolo/
!cp -r "/content/drive/MyDrive/Kobot/YOLO/darknet/weights" ./weights
%ls 

# Tset YOLO

In [ ]:
!./darknet detect cfg/yolov4.cfg weights/yolov4.weights data/dog.jpg

In [ ]:
#학습된 파일은 predictions.jpg 이라는 이름으로 저장된다. 
imShow('predictions.jpg')